# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(528, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.25,73,9,5.00,GS,1707101077
1,1,badger,64.8000,-147.5333,-20.36,74,100,0.00,US,1707101100
2,2,la ronge,55.1001,-105.2842,-7.86,86,100,2.57,CA,1707101296
3,3,portland,45.5234,-122.6762,11.56,58,100,4.92,US,1707100800
4,4,adamstown,-25.0660,-130.1015,25.17,79,100,3.73,PN,1707101297


In [3]:
city_data_df.sort_values("Humidity").head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
464,464,ciprian porumbescu,47.5667,26.0667,7.10,10,99,4.42,RO,1707101543
290,290,deqen,29.9618,90.7188,-0.08,11,0,5.04,CN,1707101443
351,351,alice springs,-23.7000,133.8833,43.45,11,0,2.24,AU,1707101361
141,141,kiiminki,65.1314,25.7974,-5.19,14,99,4.45,FI,1707101368
445,445,sabinas,27.8500,-101.1167,18.69,15,0,5.19,MX,1707101533


In [4]:
city_data_df.sort_values("Humidity").tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
248,248,kapuskasing,49.4169,-82.4331,-2.95,100,0,1.54,CA,1707101086
209,209,minot,48.2325,-101.2963,3.12,100,100,4.12,US,1707101280
254,254,beloyarskiy,63.7119,66.6722,-29.44,100,95,1.34,RU,1707101375
480,480,zaragoza,41.6561,-0.8773,8.90,100,100,1.54,ES,1707101109
334,334,ondjiva,-17.0667,15.7333,21.02,100,20,0.51,AO,1707101467


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df['Max Temp'] > 20) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 5) & 
    (city_data_df['Cloudiness'] == 0)
].copy()

# Drop any rows with null values
criteria_df.dropna(inplace=True)

# Display sample data
criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,east london,-33.0153,27.9116,21.94,74,0,1.77,ZA,1707101081
63,63,coahuayana de hidalgo,18.7000,-103.6583,20.20,79,0,1.76,MX,1707101324
67,67,cabo san lucas,22.8909,-109.9124,22.09,70,0,0.45,MX,1707101326
93,93,brisas de zicatela,15.8369,-97.0419,24.64,67,0,3.49,MX,1707101344
177,177,port isabel,26.0734,-97.2086,21.02,33,0,4.63,US,1707101283


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df.shape)
hotel_df.head()

(528, 6)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,73,
1,badger,US,64.8000,-147.5333,74,
2,la ronge,CA,55.1001,-105.2842,86,
3,portland,US,45.5234,-122.6762,58,
4,adamstown,PN,-25.0660,-130.1015,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search a hotel
radius = 10000
category = 'accommodation.hotel'  
apiKey = geoapify_key  

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        'filter': f'category:{category}',
        'bias': f'proximity:{lng},{lat}',
        'apiKey': geoapify_key,
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.url}")
        continue
      
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-36.5092%2C-54.2811&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-147.5333%2C64.8&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-105.2842%2C55.1001&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-122.6762%2C45.5234&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-130.1015%2C-25.066&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A70.2167%2C-49.35&apiKey=d3c40354a61e4db595676b0f801de438
Error:

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A132.95%2C48.8&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A12.9801%2C31.4451&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A145.701%2C15.1355&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A86.1778%2C69.4058&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-0.8967%2C45.5549&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A27.9116%2C-33.0153&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-98.2023%2C32.2207&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-77.6267%2C-11.0925&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-24.7167%2C14.8667&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-34.8086%2C-7.4706&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-52.7813%2C47.5166&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A22.45%2C-26.05&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapi

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A8.0602%2C34.7449&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A22.6814%2C-14.97&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A119.9001%2C-2.9627&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-175.2%2C-21.1333&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A2.4607%2C27.1935&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-51.995%2C-6.6447&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.co

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-53.6735%2C66.9395&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-28.6333%2C38.5333&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-71.6273%2C-33.0393&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-83.6807%2C10.2104&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A151.2833%2C59.5833&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-101.2963%2C48.2325&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.g

...

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-71.4639%2C-43.0858&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A163.0078%2C5.3248&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A126.3658%2C-8.8673&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A168.3219%2C-17.7338&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A42.7394%2C13.0092&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A25.4582%2C50.8343&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoa

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A56.2757%2C38.9764&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A85.6667%2C53.0833&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A8.77%2C47.4765&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A15.3918%2C67.2588&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-49.9317%2C-28.2939&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-59.6997%2C-2.6994&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A125.4048%2C12.2645&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-44.8683%2C-1.8283&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-88.7333%2C15.05&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-107.9833%2C31.1&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-112.15%2C30.7167&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A121.9702%2C20.4487&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A55.4781%2C-21.3393&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-116.9754%2C46.3804&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-40.2925%2C-20.3297&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A34.3429%2C66.6871&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A20.5965%2C32.19&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A26.0667%2C47.5667&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapi

Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-6.3865%2C58.2093&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A12.6316%2C66.0217&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-155.9522%2C19.6228&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A25.0865%2C31.7561&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A167.1667%2C-15.5333&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapify.com/v2/places?filter=category%3Aaccommodation.hotel&bias=proximity%3A-27.2167%2C38.65&apiKey=d3c40354a61e4db595676b0f801de438
Error: 400 - https://api.geoapi

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)
  
# Display the map
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

In [10]:
# https://plotly.com/python/map-configuration/
# https://www.w3schools.com/python/ref_list_copy.asp
# https://apidocs.geoapify.com/docs/places/#about
#https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy